# RT Delay Analysis for ITP ID 350

In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_filter_map_plot

In [2]:
## parameters cell
itp_id = 300

In [3]:
# Parameters
itp_id = 350


In [4]:
analysis_date = dt.date(2022, 3, 30) ##TODO shift to Mar 30 for all?

In [5]:
trips = pd.read_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{itp_id}_03_30.parquet')

In [6]:
stop_delay = gpd.read_parquet(f'{utils.GCS_FILE_PATH}stop_delay_views/{itp_id}_03_30.parquet')
stop_delay['arrival_time'] = stop_delay.arrival_time.map(lambda x: dt.datetime.fromisoformat(x))
stop_delay['actual_time'] = stop_delay.actual_time.map(lambda x: dt.datetime.fromisoformat(x))

In [7]:
%%capture
routelines = utils.get_routelines(itp_id, analysis_date)

In [8]:
%%capture
rt_day = rt_filter_map_plot.RtFilterMapper(trips, stop_delay, routelines)

In [9]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00')

In [10]:
## avoid papermill errors when map fails to render
m = None

In [11]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds')

In [12]:
m

In [13]:
%%capture
rt_day.set_filter(start_time='07:00', end_time='09:00')

In [14]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds')

In [15]:
m

In [16]:
try:
    metrics_df = (rt_day.rt_trips
         >> group_by(_.route_short_name, _.direction_id)
         >> summarize(speed_variance = _.mean_speed_mph.var(),
                     mean_speed_mph = _.mean_speed_mph.mean(),
                     min_hour = _.median_time.min().hour,
                     max_hour = _.median_time.max().hour)
         >> filter(_.min_hour <= 7, _.max_hour >= 15,
                   _.mean_speed_mph < _.mean_speed_mph.quantile(.6),
                   _.speed_variance)
         >> arrange(-_.speed_variance)
         >> head(20)
        )
    metrics_df
except Exception as e:
    print(e)
## check that route has all-day span...

In [17]:
def map_from_metrics(rt_day, metrics_df):
    try:
        route_name = metrics_df.iloc[1,0]
        direction_id = metrics_df.iloc[1,1]
    except Exception as e:
        print(e)
        return
    # route_name = '281'
    # direction_id = '1'
    rt_day.set_filter(start_time='07:00', end_time='09:00',
                      route_names=[route_name], direction_id = direction_id)
    try:
        am_map = rt_day.segment_speed_map(how='low_speeds')
        display(am_map)
    except Exception as e:
        print(e)
        pass
    
    rt_day.set_filter(start_time='11:00', end_time='14:00',
                      route_names=[route_name], direction_id = direction_id)
    try:
        mid_map = rt_day.segment_speed_map(how='low_speeds')
        display(mid_map)
    except Exception as e:
        print(e)
        pass

    rt_day.set_filter(start_time='15:00', end_time='19:00',
                      route_names=[route_name], direction_id = direction_id)
    try:
        pm_map = rt_day.segment_speed_map(how='low_speeds')
        display(pm_map)
    except Exception as e:
        print(e)
        pass

    return

In [18]:
try:
    map_from_metrics(rt_day, metrics_df)
except Exception as e:
    print(e)
    pass

speed above 80 for shape p_1274523, dropping
